# Hyperparameter tuning (Spark based recommender)

Hyperparameter tuning for Spark based recommender algorithm is important to select a model with the optimal performance. This notebook introduces good practices in performing hyperparameter tuning for building recommender models with the utility functions provided in the [Microsoft/Recommenders](https://github.com/Microsoft/Recommenders.git) repository.

Three different approaches are introduced and comparatively studied.
* Spark native constructs (`ParamGridBuilder`, `TrainValidationSplit`).
* `hyperopt` package with Tree of Parzen Estimator algorithm. 
* Parallelized random search of parameter values sampled with pre-defined space. 

## 0 Global settings and import

In [1]:
# set the environment path to find Recommenders
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import sys
sys.path.append("../../")
import pandas as pd
import numpy as np
import time

import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import RegressionEvaluator

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope
from hyperopt.pyll.stochastic import sample

from reco_utils.common.spark_utils import start_or_get_spark
from reco_utils.evaluation.spark_evaluation import SparkRankingEvaluation, SparkRatingEvaluation
from reco_utils.dataset.movielens import load_spark_df
from reco_utils.dataset.spark_splitters import spark_random_split

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("PySpark version: {}".format(pyspark.__version__))

System version: 3.6.0 | packaged by conda-forge | (default, Feb  9 2017, 14:36:55) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]
Pandas version: 0.23.4
PySpark version: 2.3.1


In [2]:
%env PYSPARK_PYTHON=/anaconda/envs/recommender/bin/python
%env PYSPARK_DRIVER_PYTHON=/anaconda/envs/recommender/bin/python

env: PYSPARK_PYTHON=/anaconda/envs/recommender/bin/python
env: PYSPARK_DRIVER_PYTHON=/anaconda/envs/recommender/bin/python


In [3]:
COL_USER = "UserId"
COL_ITEM = "MovieId"
COL_TIMESTAMP = "Timestamp"
COL_RATING = "Rating"
COL_PREDICTION = "prediction"

HEADER = {
    "col_user": COL_USER,
    "col_item": COL_ITEM,
    "col_rating": COL_RATING,
    "col_prediction": COL_PREDICTION,
}

HEADER_ALS = {
    "userCol": COL_USER,
    "itemCol": COL_ITEM,
    "ratingCol": COL_RATING
}

## 1 Data preparation

For illustration purpose, data used for demonstrating the hyperparameter tuning practices is the Movielens 100k dataset. 

In [4]:
spark = (
    SparkSession.builder.appName("Hyperopt")
    .master('local[*]')
    .config("spark.jars.packages", "Azure:mmlspark:0.15")
    .config("spark.driver.memory", '16g')
    .getOrCreate()
)

Movielens 100k dataset is used for running the demonstration.

In [22]:
data = load_spark_df(spark, size='100k')

The dataset is split into 3 subsets randomly with a given split ratio. The hyperparameter tuning is performed on the training and the validating data, and then the optimal recommender selected is evaluated on the testing dataset.

In [23]:
train, valid, test = spark_random_split(data, ratio=[3, 1, 1])

## 2 Hyper parameter tuning with Azure Machine Learning Services

The `hyperdrive` module in the [Azure Machine Learning Services](https://azure.microsoft.com/en-us/services/machine-learning-service/) is supposed to run [hyperparameter tuning and optimizing for machine learning model selection](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters). At the moment, the service supports running hyperparameter tuning on heterogenous computing targets such as cluster of commodity compute nodes with or without GPU devices (see detailed documentation [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets)). It is feasible to run parameter tuning on a cluster of VM nodes. In this case, the service containerizes individual and independent Spark session on each node of the cluster to run the parameter tuning job in parallel, instead of inside a single Spark session where the training is executed in a distributed manner.  

It is worth mentioning that tuning hyperparameter with `hyperdrive` in Azure Machine Learning services will be a feature in the future version. 

Detailed instructions of tuning hyperparameter of non-Spark workloads by using Azure Machine Learning Services can be found in [this](https://github.com/Microsoft/Recommenders.git) notebook. 

## 3 Hyper parameter tuning with Spark ML constructs

### 3.1 Spark native construct

Spark ML lib implements modules such as `CrossValidator` and `TrainValidationSplit` for tuning hyperparameters (see [here](https://spark.apache.org/docs/2.2.0/ml-tuning.html)). However, by default, it does not support custom machine learning algorithms, data splitting methods, and evaluation metrics, like what are offered as utility functions in the Recommenders repository. 

For example, the Spark native constuct can be used for tuning a recommender against the `rmse` metric which is one of the available regression metrics in Spark.

Firstly, a Spark ALS object needs to be created. In this case, for illustration purpose, it is an ALS model object.

In [7]:
# NOTE the parameters of interest, rank and regParam, are left unset, 
# because their values will be assigned in the parameter grid builder.
als = ALS(
    maxIter=15,
    implicitPrefs=False,
    alpha=0.1,
    coldStartStrategy='drop',
    nonnegative=False,
    **HEADER_ALS
)

Then, a parameter grid can be defined as follows. Without loss of generity, only `rank` and `regParam` are considered.

In [8]:
paramGrid = ParamGridBuilder() \
    .addGrid(als.rank, [10, 20]) \
    .addGrid(als.regParam, [ 0.1, 0.01, 0.001, 0.0001, 0.00001]) \
    .build()

Given the settings above, a `TrainValidationSplit` constructor can be created for fitting the best model in the given parameter range.

In [9]:
tvs = TrainValidationSplit(
    estimator=als,
    estimatorParamMaps=paramGrid,
    # A regression evaluation method is used. 
    evaluator=RegressionEvaluator(labelCol='Rating'),
    # 80% of the data will be used for training, 20% for validation.
    # NOTE here the splitting is random. The Spark splitting utilities (e.g. chrono splitter)
    # are therefore not available here. 
    trainRatio=0.75
)

In [10]:
time_start = time.time()

# Run TrainValidationSplit, and choose the best set of parameters.
# NOTE train and valid is union because in Spark TrainValidationSplit does splitting by itself.
model = tvs.fit(train.union(valid))

time_spark = time.time() - time_start

The model parameters in the grid and the best metrics can be then returned. 

In [11]:
for idx, item in enumerate(model.getEstimatorParamMaps()):
    print('Run {}:'.format(idx))
    print('\tValidation Metric: {}'.format(model.validationMetrics[idx]))
    for key, value in item.items():
        print('\t{0}: {1}'.format(repr(key), value))

Run 0:
	Validation Metric: 0.9421215576763626
	Param(parent='ALS_4a6d87a44ea816ff815b', name='rank', doc='rank of the factorization'): 10
	Param(parent='ALS_4a6d87a44ea816ff815b', name='regParam', doc='regularization parameter (>= 0).'): 0.1
Run 1:
	Validation Metric: 1.1935485782392687
	Param(parent='ALS_4a6d87a44ea816ff815b', name='rank', doc='rank of the factorization'): 10
	Param(parent='ALS_4a6d87a44ea816ff815b', name='regParam', doc='regularization parameter (>= 0).'): 0.01
Run 2:
	Validation Metric: 1.536557434836125
	Param(parent='ALS_4a6d87a44ea816ff815b', name='rank', doc='rank of the factorization'): 10
	Param(parent='ALS_4a6d87a44ea816ff815b', name='regParam', doc='regularization parameter (>= 0).'): 0.001
Run 3:
	Validation Metric: 2.108594860151324
	Param(parent='ALS_4a6d87a44ea816ff815b', name='rank', doc='rank of the factorization'): 10
	Param(parent='ALS_4a6d87a44ea816ff815b', name='regParam', doc='regularization parameter (>= 0).'): 0.0001
Run 4:
	Validation Metric: 2

In [12]:
model.validationMetrics

[0.9421215576763626,
 1.1935485782392687,
 1.536557434836125,
 2.108594860151324,
 2.6397158225049893,
 0.9418574694424408,
 1.2875617383316302,
 1.6891997782841075,
 2.215253494308962,
 3.1110708371552813]

To get the best model, just do

In [13]:
model_best_spark = model.bestModel

### 3.2 Custom `Estimator`, `Transformer`, and `Evaluator` for Spark ALS

One can also customize Spark modules to allow tuning hyperparameters for a desired model. For instance, the native Spark ALS does not allow tuning hyperparameters for ranking metrics such as precision@k, recall@k, etc. This can be done by creating custom `Estimator`, `Transformer` and `Evaluator`. The benefit is that, after the customization, the tuning process can make use of `trainValidSplit` directly, which distributes the tuning in a Spark session.

#### Customized `Estimator` and `Transformer` for top k recommender based on Spark ALS

In [9]:
class ALSTopK(
    ALS,
    Estimator,
    HasInputCol,
    HasPredictionCol
):    
    rank = Param(Params._dummy(), "rank", "rank of the factorization",
                 typeConverter=TypeConverters.toInt)
    numUserBlocks = Param(Params._dummy(), "numUserBlocks", "number of user blocks",
                          typeConverter=TypeConverters.toInt)
    numItemBlocks = Param(Params._dummy(), "numItemBlocks", "number of item blocks",
                          typeConverter=TypeConverters.toInt)
    implicitPrefs = Param(Params._dummy(), "implicitPrefs", "whether to use implicit preference",
                          typeConverter=TypeConverters.toBoolean)
    alpha = Param(Params._dummy(), "alpha", "alpha for implicit preference",
                  typeConverter=TypeConverters.toFloat)
    userCol = Param(Params._dummy(), "userCol", "column name for user ids. Ids must be within " +
                    "the integer value range.", typeConverter=TypeConverters.toString)
    itemCol = Param(Params._dummy(), "itemCol", "column name for item ids. Ids must be within " +
                    "the integer value range.", typeConverter=TypeConverters.toString)
    ratingCol = Param(Params._dummy(), "ratingCol", "column name for ratings",
                      typeConverter=TypeConverters.toString)
    nonnegative = Param(Params._dummy(), "nonnegative",
                        "whether to use nonnegative constraint for least squares",
                        typeConverter=TypeConverters.toBoolean)
    intermediateStorageLevel = Param(Params._dummy(), "intermediateStorageLevel",
                                     "StorageLevel for intermediate datasets. Cannot be 'NONE'.",
                                     typeConverter=TypeConverters.toString)
    finalStorageLevel = Param(Params._dummy(), "finalStorageLevel",
                              "StorageLevel for ALS model factors.",
                              typeConverter=TypeConverters.toString)
    coldStartStrategy = Param(Params._dummy(), "coldStartStrategy", "strategy for dealing with " +
                              "unknown or new users/items at prediction time. This may be useful " +
                              "in cross-validation or production scenarios, for handling " +
                              "user/item ids the model has not seen in the training data. " +
                              "Supported values: 'nan', 'drop'.",
                              typeConverter=TypeConverters.toString)

    @keyword_only
    def __init__(
        self,
        rank=10, maxIter=10, regParam=0.1, numUserBlocks=10, numItemBlocks=10,
        implicitPrefs=False, alpha=1.0, userCol="user", itemCol="item", seed=None, k=10,
        ratingCol="rating", nonnegative=False, checkpointInterval=10,
        intermediateStorageLevel="MEMORY_AND_DISK",
        finalStorageLevel="MEMORY_AND_DISK", coldStartStrategy="nan"
    ):
        super(ALS, self).__init__()
        self._java_obj = self._new_java_obj("org.apache.spark.ml.recommendation.ALS", self.uid)
        self._setDefault(rank=10, maxIter=10, regParam=0.1, numUserBlocks=10, numItemBlocks=10,
                         implicitPrefs=False, alpha=1.0, userCol="user", itemCol="item",
                         ratingCol="rating", nonnegative=False, checkpointInterval=10,
                         intermediateStorageLevel="MEMORY_AND_DISK",
                         finalStorageLevel="MEMORY_AND_DISK", coldStartStrategy="nan")

        kwargs = self._input_kwargs 
        kwargs = {x: kwargs[x] for x in kwargs if x not in {'k'}}
        self.setParams(**kwargs)
        
        # The manually added parameter is not present in ALS Java implementation. 
        self.k = k
        
    def setRank(self, value):
        """
        Sets the value of :py:attr:`rank`.
        """
        return self._set(rank=value)

    def getRank(self):
        """
        Gets the value of rank or its default value.
        """
        return self.getOrDefault(self.rank)
    
    def setParams(self, rank=10, maxIter=10, regParam=0.1, numUserBlocks=10, numItemBlocks=10,
                  implicitPrefs=False, alpha=1.0, userCol="user", itemCol="item", seed=None,
                  ratingCol="rating", nonnegative=False, checkpointInterval=10,
                  intermediateStorageLevel="MEMORY_AND_DISK",
                  finalStorageLevel="MEMORY_AND_DISK", coldStartStrategy="nan"):
        """
        setParams(self, rank=10, maxIter=10, regParam=0.1, numUserBlocks=10, numItemBlocks=10, \
                 implicitPrefs=False, alpha=1.0, userCol="user", itemCol="item", seed=None, \
                 ratingCol="rating", nonnegative=False, checkpointInterval=10, \
                 intermediateStorageLevel="MEMORY_AND_DISK", \
                 finalStorageLevel="MEMORY_AND_DISK", coldStartStrategy="nan")
        Sets params for ALS.
        """
        kwargs = self._input_kwargs
        kwargs = {x: kwargs[x] for x in kwargs if x not in {'k'}}
        return self._set(**kwargs)
        
    def _fit(self, dataset):
        kwargs = self._input_kwargs    
        # Exclude k as it is not a parameter for ALS.
        kwargs = {x: kwargs[x] for x in kwargs if x not in {'k'}}
        kwargs['rank'] = self.getRank()
        kwargs['regParam'] = self.getOrDefault(self.regParam)
        als = ALS(
            **kwargs
        )
        als_model = als.fit(dataset)
        
        user_col = kwargs['userCol']
        item_col = kwargs['itemCol']
        
        k = self.k
                     
        topk_model = ALSTopKModel()
        topk_model.setParams(
            als_model,
            user_col, 
            item_col, 
            k
        )
        
        return topk_model
    
    
class ALSTopKModel(
    Model,
    HasInputCol,
    HasPredictionCol,
    HasLabelCol
):    
    def setParams(self, model, userCol, itemCol, k):
        self.model = model
        self.userCol = userCol
        self.itemCol = itemCol
        self.k = k
    
    def _transform(self, dataset):
        predictionCol = self.getPredictionCol()
        labelCol = self.getLabelCol()
        
        users = dataset.select(self.userCol).distinct()
        topk_recommendation = self.model.recommendForUserSubset(users, self.k)  
        
        extract_value = F.udf((lambda x: [y[0] for y in x]), ArrayType(IntegerType()))
        topk_recommendation = topk_recommendation.withColumn(predictionCol, extract_value(F.col("recommendations")))        
        
        dataset = (
            dataset
            .groupBy(self.userCol)
            .agg(F.collect_list(F.col(self.itemCol)).alias(labelCol))
        )
            
        topk_recommendation_all = dataset.join(
            topk_recommendation, 
            on=self.userCol,
            how="outer"
        )
        
        return topk_recommendation_all.select(self.userCol, labelCol, predictionCol)

#### Customized precision@k evaluation metric

In [10]:
from pyspark.ml.evaluation import Evaluator
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics

# Define a custom Evaulator. Here precision@k is used.
class PrecisionAtKEvaluator(Evaluator):

    def __init__(self, predictionCol="prediction", labelCol="label", k=10):
        self.predictionCol = predictionCol
        self.labelCol = labelCol
        self.k = k

    def _evaluate(self, dataset):
        """
        Returns a random number. 
        Implement here the true metric
        """      
        # Drop Nulls.
        dataset = dataset.na.drop()
        metrics = RankingMetrics(dataset.select(self.predictionCol, self.labelCol).rdd)
        return metrics.precisionAt(self.k)

    def isLargerBetter(self):
        return True

Create new ALS top-k recommender and find the optimal model with a train-valid splitter.

In [18]:
alstopk = ALSTopK(
    userCol="UserId",
    itemCol="MovieId",
    ratingCol="Rating",
    k=10
)

paramGrid = ParamGridBuilder() \
    .addGrid(alstopk.rank, [10, 20]) \
    .addGrid(alstopk.regParam, [0.1, 0.01, 0.001, 0.0001, 0.00001]) \
    .build()

tvs = TrainValidationSplit(
    estimator=alstopk,
    estimatorParamMaps=paramGrid,
    # A regression evaluation method is used. 
    evaluator=PrecisionAtKEvaluator(),
    # 80% of the data will be used for training, 20% for validation.
    # NOTE here the splitting is random. The Spark splitting utilities (e.g. chrono splitter)
    # are therefore not available here. 
    trainRatio=0.75
)

In [31]:
time_start = time.time()

# Run TrainValidationSplit, and choose the best set of parameters.
# NOTE train and valid is union because in Spark TrainValidationSplit does splitting by itself.
model = tvs.fit(train.union(valid))

time_spark = time.time() - time_start

{'userCol': 'UserId', 'itemCol': 'MovieId', 'ratingCol': 'Rating', 'rank': 10, 'regParam': 0.1}
{'userCol': 'UserId', 'itemCol': 'MovieId', 'ratingCol': 'Rating', 'rank': 10, 'regParam': 0.01}
{'userCol': 'UserId', 'itemCol': 'MovieId', 'ratingCol': 'Rating', 'rank': 10, 'regParam': 0.001}
{'userCol': 'UserId', 'itemCol': 'MovieId', 'ratingCol': 'Rating', 'rank': 10, 'regParam': 0.0001}
{'userCol': 'UserId', 'itemCol': 'MovieId', 'ratingCol': 'Rating', 'rank': 10, 'regParam': 1e-05}
{'userCol': 'UserId', 'itemCol': 'MovieId', 'ratingCol': 'Rating', 'rank': 20, 'regParam': 0.1}
{'userCol': 'UserId', 'itemCol': 'MovieId', 'ratingCol': 'Rating', 'rank': 20, 'regParam': 0.01}
{'userCol': 'UserId', 'itemCol': 'MovieId', 'ratingCol': 'Rating', 'rank': 20, 'regParam': 0.001}
{'userCol': 'UserId', 'itemCol': 'MovieId', 'ratingCol': 'Rating', 'rank': 20, 'regParam': 0.0001}
{'userCol': 'UserId', 'itemCol': 'MovieId', 'ratingCol': 'Rating', 'rank': 20, 'regParam': 1e-05}
{'userCol': 'UserId', 'i

In [32]:
time_spark

98.03197407722473

In [38]:
model.getEstimatorParamMaps()

[{Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='rank', doc='rank of the factorization'): 10,
  Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='regParam', doc='regularization parameter (>= 0).'): 0.1},
 {Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='rank', doc='rank of the factorization'): 10,
  Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='regParam', doc='regularization parameter (>= 0).'): 0.01},
 {Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='rank', doc='rank of the factorization'): 10,
  Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='regParam', doc='regularization parameter (>= 0).'): 0.001},
 {Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='rank', doc='rank of the factorization'): 10,
  Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='regParam', doc='regularization parameter (>= 0).'): 0.0001},
 {Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='rank', doc='rank of the factorization'): 10,
  Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='regPara

In [66]:
def best_param(model, is_larger_better=True):
    if is_larger_better:
        best_metric = max(model.validationMetrics)
    else:
        best_metric = min(model.validationMetrics)
        
    parameters = model.getEstimatorParamMaps()[model.validationMetrics.index(best_metric)]
     
    return list(parameters.values())

In [62]:
params = best_param(model)

In [47]:
model.bestModel.transform(valid).show()

+------+--------------------+--------------------+
|UserId|               label|          prediction|
+------+--------------------+--------------------+
|   148|[7, 8, 78, 127, 1...|[408, 173, 169, 5...|
|   463|[1, 7, 16, 19, 10...|[613, 116, 19, 88...|
|   471|[50, 99, 404, 420...|[1242, 102, 477, ...|
|   496|[28, 50, 53, 133,...|[320, 1639, 114, ...|
|   833|[23, 26, 72, 92, ...|[1597, 1187, 1070...|
|   243|[13, 22, 83, 194,...|[1642, 1398, 1449...|
|   392|[98, 99, 174, 189...|[1463, 408, 483, ...|
|   540|[7, 20, 25, 257, ...|[1449, 1019, 745,...|
|   623|[163, 211, 234, 2...|[483, 496, 318, 5...|
|   737|[89, 169, 174, 18...|[1405, 119, 1103,...|
|   858|     [323, 334, 515]|[276, 1642, 1193,...|
|   897|[11, 23, 66, 68, ...|[1643, 1169, 64, ...|
|    31|[124, 299, 319, 3...|[1463, 175, 1643,...|
|   516|[194, 204, 214, 431]|[1062, 1137, 1642...|
|    85|[10, 13, 14, 30, ...|[1463, 408, 1398,...|
|   137|[183, 210, 257, 3...|[1019, 1169, 1643...|
|   251|[24, 60, 79, 117,...|[1

In [33]:
for idx, item in enumerate(model.getEstimatorParamMaps()):
    print('Run {}:'.format(idx))
    print('\tValidation Metric: {}'.format(model.validationMetrics[idx]))
    for key, value in item.items():
        print('\t{0}: {1}'.format(repr(key), value))

Run 0:
	Validation Metric: 0.030286928799149855
	Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='rank', doc='rank of the factorization'): 10
	Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='regParam', doc='regularization parameter (>= 0).'): 0.1
Run 1:
	Validation Metric: 0.011477151965993623
	Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='rank', doc='rank of the factorization'): 10
	Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='regParam', doc='regularization parameter (>= 0).'): 0.01
Run 2:
	Validation Metric: 0.00563230605738576
	Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='rank', doc='rank of the factorization'): 10
	Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='regParam', doc='regularization parameter (>= 0).'): 0.001
Run 3:
	Validation Metric: 0.004994686503719443
	Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='rank', doc='rank of the factorization'): 10
	Param(parent='ALSTopK_4fee9bc0af0538c7d0f3', name='regParam', doc='regularization parameter (>= 0

### 3.3 `mmlspark`?

In [24]:
from mmlspark import HyperparamBuilder
from mmlspark import RangeHyperParam
from mmlspark import DiscreteHyperParam
from mmlspark import RandomSpace

alstopk = ALSTopK(
    userCol="UserId",
    itemCol="MovieId",
    ratingCol="Rating",
    k=10
)

paramBuilder = (
    HyperparamBuilder()
    .addHyperparam(alstopk, alstopk.rank, DiscreteHyperParam([10, 15]))
    .addHyperparam(alstopk, alstopk.regParam, RangeHyperParam(0.1, 0.3, isDouble=True))
)

paramSpace = paramBuilder.build()
randomSpace = RandomSpace(paramBuilder.build())

In [27]:
tvs = TrainValidationSplit(
    estimator=alstopk,
    estimatorParamMaps=randomSpace,
    # A regression evaluation method is used. 
    evaluator=PrecisionAtKEvaluator(),
    # 80% of the data will be used for training, 20% for validation.
    # NOTE here the splitting is random. The Spark splitting utilities (e.g. chrono splitter)
    # are therefore not available here. 
    trainRatio=0.75
)

tvs.fit(train)

TypeError: object of type 'RandomSpace' has no len()

In [31]:
from mmlspark import TuneHyperparameters

bestModel = TuneHyperparameters(
    evaluationMetric="accuracy", 
    models=alstopk, 
    numFolds=2,
    numRuns=1, 
    parallelism=1,
    paramSpace=randomSpace.space(), 
    seed=0
).fit(train)

Py4JJavaError: An error occurred while calling o2206.fit.
: java.lang.ClassCastException: org.apache.spark.ml.recommendation.ALS cannot be cast to [Lorg.apache.spark.ml.Estimator;
	at com.microsoft.ml.spark.TuneHyperparameters.getModels(TuneHyperparameters.scala:43)
	at com.microsoft.ml.spark.TuneHyperparameters.fit(TuneHyperparameters.scala:119)
	at com.microsoft.ml.spark.TuneHyperparameters.fit(TuneHyperparameters.scala:33)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.

## 4 Hyperparameter tuning with `hyperopt`

`hyperopt` is an open source Python package that is designed for tuning parameters for generic function with any pre-defined loss. More information about `hyperopt` can be found [here](https://github.com/hyperopt/hyperopt). `hyperopt` supports parallelization on MongoDB but not Spark. In our case, the tuning is performed in a sequential mode on a local computer.

In `hyperopt`, an *objective* function is defined for optimizing the hyper parameters. In this case, the objective is similar to that in the Spark native construct situation, which is *to the RMSE metric for an ALS recommender*. Parameters of `rank` and `regParam` are used as hyperparameters. 

The objective function shown below demonstrates a RMSE loss for an ALS recommender. 

In [ ]:
# Customize an objective function
def objective(params):
    rank = params['rank']
    reg = params['reg']
    train = params['train'] 
    valid = params['valid'] 
    col_user = params['col_user'] 
    col_item = params['col_item']
    col_rating = params['col_rating'] 
    col_prediction = params['col_prediction'] 
    k = params['k']
    relevancy_method = params['relevancy_method']
    
    als = ALS(
        rank=rank,
        maxIter=15,
        implicitPrefs=False,
        alpha=0.1,
        regParam=reg,
        coldStartStrategy='drop',
        nonnegative=False,
        seed=0,
        **HEADER_ALS
    )
    
    model = als.fit(train) 
    prediction = model.transform(valid)

    rating_eval = SparkRatingEvaluation(
        valid, 
        prediction, 
        **HEADER
    )
    
    rmse = rating_eval.rmse()
    
    # Return the objective function result.
    return {
        'loss': rmse,
        'status': STATUS_OK,
        'eval_time': time.time()
    }

A search space is usually defined for hyperparameter exploration. Design of search space is empirical, and depends on the understanding of how distribution of parameter of interest affects the model performance measured by the loss function. 

In the ALS algorithm, the two hyper parameters, rank and reg, affect model performance in a way that
* The higher the rank, the better the model performance but also the higher risk of overfitting.
* The reg parameter is subject to rank and it prevents overfitting in certain way. 

Therefore, in this case, a uniform distribution and a lognormal distribution sampling spaces are used for rank and reg, respectively. A narrow search space is used for illustration purpose, that is, the range of rank is from 10 to 20, while that of reg is from $e^{-5}$ to $e^{-1}$. Together with the randomly sampled hyper parameters, other parameters use for building / evaluating the recommender, like `k`, column names, data, etc., are kept as constants.

In [ ]:
# define a search space
space = {
    'rank': hp.quniform('rank', 10, 20, 5),
    'reg': hp.loguniform('reg', -5, -1),
    'train': train, 
    'valid': valid, 
    'col_user': "UserId", 
    'col_item': "MovieId", 
    'col_rating': "Rating", 
    'col_prediction': "prediction", 
    'k': 10,
    'relevancy_method': "top_k"
}

`fmin` of `hyperopt` is used for running the trials for searching optimal hyper parameters. In `hyperopt`, there are different strategies for intelligently optimize hyper parameters. For example, `hyperopt` avails [Tree of Parzen Estimators (TPE) method](https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf) for searching optimal parameters, which is more efficient than random search or grid search (details can be found [here](http://hyperopt.github.io/hyperopt/)). It is claimed in the project website that Bayesian based optimization method is planned but so far it has not yet been implemented. 

The following runs the trials with the pre-defined objective function and search space. TPE is used as the optimization method. Totally there will be 10 evaluations run for searching the best parameters.

In [ ]:
time_start = time.time()

# Trials for recording each iteration of the hyperparameter searching.
trials = Trials()

best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    trials=trials,
    max_evals=10
)
                  
time_hyperopt = time.time() - time_start

In [ ]:
trials.best_trial

In [ ]:
parameters = ['rank', 'reg']
cols = len(parameters)
f, axes = plt.subplots(nrows=1, ncols=cols, figsize=(15,5))
cmap = plt.cm.jet
for i, val in enumerate(parameters):
    xs = np.array([t['misc']['vals'][val] for t in trials.trials]).ravel()
    ys = [t['result']['loss'] for t in trials.trials]
    xs, ys = zip(*sorted(zip(xs, ys)))
    ys = np.array(ys)
    axes[i].scatter(xs, ys, s=20, linewidth=0.01, alpha=0.75, c=cmap(float(i)/len(parameters)))
    axes[i].set_title(val)

It can be seen from the above plot that
* The actual impact of rank is in line with the intuition - the larger the value the better the result.
* It is interesting to see that the optimal value of reg is ~0.1. 

Get the best model.

In [ ]:
als = ALS(
    rank=best["rank"],
    regParam=best["reg"],
    maxIter=15,
    implicitPrefs=False,
    alpha=0.1,
    coldStartStrategy='drop',
    nonnegative=False,
    seed=0,
    **HEADER_ALS
)
    
model_best_hyperopt = als.fit(train)

Tuning prameters against other metrics can be simply done by modifying the `objective` function. The following shows an objective function of how to tune "precision@k". Since `fmin` in `hyperopt` only supports minimization while the actual objective of the loss is to maximize "precision@k", `-precision` instead of `precision` is used in the returned value of the `objective` function.

In [ ]:
# Customize an objective function
def objective(params):
    rank = params['rank']
    reg = params['reg']
    train = params['train'] 
    valid = params['valid'] 
    col_user = params['col_user'] 
    col_item = params['col_item']
    col_rating = params['col_rating'] 
    col_prediction = params['col_prediction'] 
    k = params['k']
    relevancy_method = params['relevancy_method']
    
    header = {
        "userCol": col_user,
        "itemCol": col_item,
        "ratingCol": col_rating,
    }
    
    als = ALS(
        rank=rank,
        maxIter=15,
        implicitPrefs=False,
        alpha=0.1,
        regParam=reg,
        coldStartStrategy='drop',
        nonnegative=False,
        seed=0,
        **header
    )
    
    model = als.fit(train)
    
    users = train.select(col_user).distinct()
    items = train.select(col_item).distinct()
    user_item = users.crossJoin(items)
    dfs_pred = model.transform(user_item)

    # Remove seen items.
    dfs_pred_exclude_train = dfs_pred.alias("pred").join(
        train.alias("train"),
        (dfs_pred[col_user] == train[col_user]) & (dfs_pred[col_item] == train[col_item]),
        how='outer'
    )

    top_all = dfs_pred_exclude_train.filter(dfs_pred_exclude_train["train.Rating"].isNull()) \
        .select('pred.' + col_user, 'pred.' + col_item, 'pred.' + "prediction")
    
    top_all.cache().count()

    rank_eval = SparkRankingEvaluation(
        valid, 
        top_all, 
        k=k, 
        col_user=col_user, 
        col_item=col_item, 
        col_rating="Rating", 
        col_prediction="prediction", 
        relevancy_method=relevancy_method
    )
    
    precision = rank_eval.precision_at_k()
    
    # Return the objective function result.
    return {
        'loss': -precision,
        'status': STATUS_OK,
        'eval_time': time.time()
    }

## 5 Hyperparameter tuning with `hyperopt` sampling methods

Though `hyperopt` works well in a single node machine, its features (e.g., `Trials` module) do not support Spark environment. A good practice is to use `hyperopt` for sampling parameter values from the defined sampling space, and then parallelize the model training onto Spark cluster with the sampled parameter combinations.

The downside of this method is that the intelligent searching algorithm (i.e., TPE) of `hyperopt` cannot be used. The approach introduced here is therefore equivalent to random search.

Sample the parameters used for model building from the pre-defined space. 

In [ ]:
import hyperopt.pyll.stochastic

time_start = time.time()

sample_params = [hyperopt.pyll.stochastic.sample(space) for x in range(10)]

If the master node of the Spark cluster has multiple cores, a local parallelization with Python `map` function can be used for running model building in parallel. Note here the Spark `flatMap` function cannot be used, because parallelizing Spark sessions in a Spark session is forbidden.

The following runs model building on the sampled parameter values with the pre-defined objective function.

In [ ]:
results_map = list(map(lambda x: objective(x), sample_params))

time_sample = time.time() - time_start

In [ ]:
results_map

Get the best model.

In [ ]:
loss_metrics = np.array([x['loss'] for x in results_map])
best_loss = np.where(loss_metrics == min(loss_metrics))

In [ ]:
best_param = sample_params[best_loss[0].item()]

In [ ]:
als = ALS(
    rank=best_param["rank"],
    regParam=best_param["reg"],
    maxIter=15,
    implicitPrefs=False,
    alpha=0.1,
    coldStartStrategy='drop',
    nonnegative=False,
    seed=0,
    **HEADER_ALS
)
    
model_best_sample = als.fit(train)

## 6 Evaluation on testing data

The optimal parameters can then be used for building a recommender, which is then evaluated on the testing data.

To compare the aforementioned three approaches, the evaluation metrics (loss) and the elapsed time to run each of them are recorded. 

In [ ]:
# Get prediction results with the optimal modesl from different approaches.
prediction_spark = model_best_spark.transform(test)
prediction_hyperopt = model_best_hyperopt.transform(test)
prediction_sample = model_best_sample.transform(test)

predictions = [prediction_spark, prediction_hyperopt, prediction_sample]
elapsed = [time_spark, time_hyperopt, time_sample]

approaches = ['spark', 'hyperopt', 'sample']
comparison = pd.DataFrame()
for ind, approach in enumerate(approaches):    
    rating_eval = SparkRatingEvaluation(
        test, 
        predictions[ind],
        **HEADER
    )
    
    result = pd.DataFrame({
        'Approach': approach,
        'RMSE': rating_eval.rmse(),
        'MAE': rating_eval.mae(),
        'Explained variance': rating_eval.exp_var(),
        'R squared': rating_eval.rsquared(),
        'Elapsed': elapsed[ind]
    }, index=[0])
    
    comparison = comparison.append(result)

In [ ]:
comparison

From the results, it can be seen that, *with the same number of iterations*, Spark native construct based approach takes the least amount of time, and not suprisingly, sample based approach takes the most amount of time. Intuitively this is because Spark native constructs leverage the underlying Java codes for running the actual analytics with high performance efficiency. There is a tradeoff in choosing between `hyperopt` TPE based approach and sampling approach in a for-loop. The former uses TPE for searching optimal parameters intelligently but runs the tuning iterations sequentially, while the latter simple samples parameter values from the pre-defined space but parallelizes the runs with Python `map` function. Run-time performance of the two approaches therefore depends on acutal use cases.  

The three approaches use the same RMSE loss. In this measure, the native Spark construct performs the best. The `hyperopt` based approach performs the second best. This may be owing to the limited number of iterations used in the TPE searching algorithm. Note the differences in the RMSE metrics may also come from the randomness of the intermediate steps in parameter tuning process. In practice, multiple runs are required for generating statistically robust comparison results. We have tried 5 times for running the same comparison codes above. The results aligned well with each other in terms of objective metric values and elapsed time. 

# Conclusions

In summary, there are mainly three different approaches for running hyperparameter tuning for Spark based recommendation algorithm. The three different approaches are compared as follows.

|Approach|Distributed (on Spark)|Param sampling|Advanced hyperparam searching algo|Custom evaluation metrics|Custom data split|
|---------|-------------|--------------|--------------------------|--------------|------------|
|AML Services|Parallelizing Spark sessions on multi-node cluster or single Spark session on one VM node.)|Random, Grid, Bayesian sampling for discrete and continuous variables.|Bandit policy, Median stopping policy, and truncation selection policy.|Yes|Yes|
|Spark native construct|Distributed in single-node standalone Spark environment or multi-node Spark cluster.|No|No|Need to re-engineer Spark modules|Need to re-engineer Spark modules.|
|`hyperopt`|No (only support parallelization on MongoDB)|Random sampling for discrete and continuous variables.|Tree Parzen Estimator|Yes|Yes|

# References

* Azure Machine Learning Services, url: https://azure.microsoft.com/en-us/services/machine-learning-service/
* Lisa Li, *et al*, Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization, The Journal of Machine Learning Research, Volume 18 Issue 1, pp 6765-6816, January 2017.
* James Bergstrat *et al*, Algorithms for Hyper-Parameter Optimization, Procs 25th NIPS 2011. 
* `hyperopt`, url: http://hyperopt.github.io/hyperopt/.
* Bergstra, J., Yamins, D., Cox, D. D. (2013) Making a Science of Model Search: Hyperparameter Optimization in Hundreds of Dimensions for Vision Architectures. Proc. of the 30th International Conference on Machine Learning (ICML 2013).
* Kris Wright, "Hyper parameter tuning with hyperopt", url:https://districtdatalabs.silvrback.com/parameter-tuning-with-hyperopt